![](CRISP_DM.png)

In [1]:
# import required packages
import subprocess
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
# set limit for displaying max amount of raws values for dataframe
pd.set_option('display.max_rows', 200)

## 2.1 Data Extraction &Understanding
* We have following three options available for extracting data....each explained in brief
    * John Hopkins (GITHUB) https://github.com/CSSEGISandData/COVID-19.git
    * RKI, webscrape (webscraping) https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html
    * REST API services to retreive data https://npgeo-corona-npgeo-de.hub.arcgis.com/

### 2.1.1 JhonsHopkins GITHUB dataset
* clonning data from Johnhopkins GITHUB page
    * using command 'git clone/pull https://github.com/CSSEGISandData/COVID-19.git'

In [2]:
# pulling data from github and storing in local drive
git_pull = subprocess.Popen('git pull', 
                     cwd = os.path.dirname( 'C:/Users/BRIJESH/ds_covid-19/data/raw/COVID-19/'), 
                     shell = True, 
                     stdout = subprocess.PIPE, 
                     stderr = subprocess.PIPE )
(out, error) = git_pull.communicate()


print("Error : " + str(error)) 
print("out : " + str(out))

NotADirectoryError: [WinError 267] The directory name is invalid

In [3]:
data_path='C:/Users/dhame/ds_covid-19/data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
pd_raw=pd.read_csv(data_path)

FileNotFoundError: [Errno 2] File C:/Users/dhame/ds_covid-19/data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv does not exist: 'C:/Users/dhame/ds_covid-19/data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'

In [4]:
pd_raw.head()

NameError: name 'pd_raw' is not defined

### 2.1.2 Webscrapping
+ RKI, webscrape (webscraping) [Robert-koch website fälle](https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html)

In [5]:
# select page by giving URL of RKI Fallzahlen
page = requests.get("https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html")

In [6]:
soup_table = BeautifulSoup(page.content, 'html.parser')

In [7]:
html_table=soup_table.find('table') # find the table, attention this works if one table exists

In [8]:
all_rows=html_table.find_all('tr')

In [9]:
final_data_list=[]
for pos,rows in enumerate(all_rows):
    col_list=[each_col.get_text(strip=True) for each_col in rows.find_all('td')] #td for data element
    final_data_list.append(col_list)

In [10]:
pd_daily_status=pd.DataFrame(final_data_list).dropna().rename(columns={0:'state',
                                                       1:'cases',
                                                       2:'changes',
                                                       3:'cases_per_100k',
                                                       4:'fatal',
                                                       5:'comment'})

In [11]:
pd_daily_status.head()

,state,cases,changes,cases_per_100k,fatal,comment
2,Baden-Württem­berg,44.903,+285,1.440,"13,0",1.867
3,Bayern,61.561,+403,2.415,"18,5",2.645
4,Berlin,12.216,+126,529,"14,1",226
5,Branden­burg,4.003,+19,66,"2,6",169
6,Bremen,2.117,+8,61,"8,9",58


### 2.1.3 REST API 
* REST API services to retreive data [NPGEO website](https://npgeo-corona-npgeo-de.hub.arcgis.com/)

In [12]:
# creating data request for Germany country
data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/Coronaf%C3%A4lle_in_den_Bundesl%C3%A4ndern/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

In [13]:
# use json package to load data from called REST API
json_object=json.loads(data.content) 
#checking data_type of Json_object
type(json_object)

dict

In [14]:
# getting keys of json file
json_object.keys()

dict_keys(['objectIdFieldName', 'uniqueIdField', 'globalIdFieldName', 'geometryProperties', 'serverGens', 'geometryType', 'spatialReference', 'fields', 'features'])

In [15]:
# create empty list name full_list and than append data from json_object
full_list=[]
for pos,each_dict in enumerate (json_object['features'][:]):
    full_list.append(each_dict['attributes'])

In [16]:
# convert full_list to pandas dataframe
pd_full_list=pd.DataFrame(full_list)
pd_full_list.head()

,OBJECTID_1,LAN_ew_AGS,LAN_ew_GEN,LAN_ew_BEZ,LAN_ew_EWZ,OBJECTID,Fallzahl,Aktualisierung,AGS_TXT,GlobalID,faelle_100000_EW,Shape__Area,Shape__Length,Death
0,1,01,Schleswig-Holstein,Land,2896712,15,4218,1599861600000,01,fc5ba936-c95c-432c-8a33-9eb2f30b660f,145.613371,4.573731e+10,2.881496e+06,161
1,2,02,Hamburg,Freie und Hansestadt,1841179,6,6770,1599861600000,02,0f3e860c-5181-4d3f-a421-1d51f50315ea,367.699175,2.089396e+09,4.188002e+05,267
2,3,03,Niedersachsen,Land,7982448,9,17783,1599861600000,03,3fd77024-c29b-4843-9be8-682ad48e60c9,222.776271,1.299836e+11,4.008988e+06,667
3,4,04,Bremen,Freie Hansestadt,682986,5,2117,1599861600000,04,4132268b-54de-4327-ac1e-760e915112f1,309.962430,1.119157e+09,3.357177e+05,58
4,5,05,Nordrhein-Westfalen,Land,17932651,10,61820,1599861600000,05,561d658f-3ee5-46e3-bc95-3528c6558ab9,344.734306,8.782936e+10,2.648673e+06,1828


In [17]:
# save dataframe to local drive in CSV format
pd_full_list.to_csv('C:/Users/dhame/ds_covid-19/data/raw/NPGEO/GER_state_data.csv',sep=';')

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/dhame/ds_covid-19/data/raw/NPGEO/GER_state_data.csv'

In [18]:
pd_full_list.describe()

,OBJECTID_1,LAN_ew_EWZ,OBJECTID,Fallzahl,Aktualisierung,faelle_100000_EW,Shape__Area,Shape__Length,Death
count,16.000000,1.600000e+01,16.000000,16.000000,1.600000e+01,16.000000,1.600000e+01,1.600000e+01,16.000000
mean,8.500000,5.188701e+06,8.500000,16155.000000,1.599862e+12,255.796310,5.672418e+10,2.142056e+06,584.187500
std,4.760952,4.936479e+06,4.760952,20729.228913,0.000000e+00,115.888049,4.556489e+10,1.258959e+06,794.135103
min,1.000000,6.829860e+05,1.000000,1055.000000,1.599862e+12,65.541181,1.119157e+09,3.357177e+05,20.000000
25%,4.750000,2.067654e+06,4.750000,3630.000000,1.599862e+12,158.455399,3.202908e+10,1.471492e+06,167.000000
50%,8.500000,3.270769e+06,8.500000,6560.500000,1.599862e+12,251.481962,5.009895e+10,2.098262e+06,226.000000
75%,12.250000,6.694969e+06,12.250000,16981.250000,1.599862e+12,337.553601,8.023081e+10,2.706878e+06,570.250000
max,16.000000,1.793265e+07,16.000000,61820.000000,1.599862e+12,470.767863,1.634855e+11,4.131181e+06,2645.000000


### 2.1.4 Additional API Access via REST Dienst, i.e INDIA Dataset
* example of REST confirm interface (Important!!!: Registration is required)
* [Smartable](https://smartable.ai/)

In [19]:
url_endpoint = 'https://api.smartable.ai/coronavirus/stats/IN'
headers = {
    'Cache-Control': 'mo-cache',
    'Subscription-Key': '22dc8d09733243328bacc2047f1c6f23'}
response = requests.get(url_endpoint, headers=headers)

ConnectionError: HTTPSConnectionPool(host='api.smartable.ai', port=443): Max retries exceeded with url: /coronavirus/stats/IN (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x00000166910BDD08>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [ ]:
print(response)

In [ ]:
# Check out what response content
response.content

In [ ]:
IN_dict = json.loads(response.content) # importing strings for India dataset and dump into JSON file with .txt format
with open ('C:/Users/dhame/ds_covid-19/data/raw/IN_data.ext','w') as outfile:
    json.dump(IN_dict, outfile,indent=2)

In [ ]:
# put all dictionary type data for INDIA into dataframe
df_4 = pd.DataFrame(IN_dict)
df_4.head()

#### 2.1.4.1 Individual States India

In [ ]:
IN_dict['stats']['breakdowns'][0]

In [ ]:
full_list_IN_country=[]
for pos,each_dict in enumerate (IN_dict['stats']['breakdowns'][:]):
    flatten_dict=each_dict['location']
    flatten_dict.update(dict(list(IN_dict['stats']['breakdowns'][pos].items())[1: 7]) 
    )
    full_list_IN_country.append(flatten_dict)

In [ ]:
df_india = pd.DataFrame(full_list_IN_country)

In [ ]:
pd.DataFrame(full_list_IN_country).to_csv('C:/Users/BRIJESH/ds_covid-19/data/raw/SMARTABLE/full_list_US_country.csv',sep=';',index=False)